In [ ]:
import pandas as pd
import graphviz
import typing
import matplotlib.pyplot as plt

from collections import defaultdict


from lib.metrics import *
from lib.message import *
from lib.analysis import PubSubAnalyzer
from lib.experiment import PubSubExperiment, PubSubExperimentResults


exp = PubSubAnalyzer.from_experiment_results(
    PubSubExperimentResults.load_from_file("experiments/kadpubsub-config1.json")
)
exp2 = PubSubAnalyzer.from_experiment_results(
    PubSubExperimentResults.load_from_file("experiments/gossipsub-config1.json")
)
exp3 = PubSubAnalyzer.from_experiment_results(
    PubSubExperimentResults.load_from_file("experiments/plumtree-config1.json")
)


In [ ]:
from typing import Dict


class PubSubPlotter:
    def __init__(self, experiments: Dict[str, PubSubAnalyzer]):
        self._experiments = experiments

    def redundancy(self):
        fig, ax = plt.subplots()
        for i, (name, exp) in enumerate(self._experiments.items()):
            exp_redundancy = exp.redundancy()
            ax.bar(name, exp_redundancy, label="Redundant Messages", color="red")
            ax.text(
                i,
                exp_redundancy / 2,
                "{0:.1f}%".format(exp_redundancy * 100),
                color="white",
                ha="center",
                va="center",
                fontweight="bold",
            )
            ax.bar(
                name,
                1 - exp_redundancy,
                bottom=exp_redundancy,
                label="Relevant Messages",
                color="green",
            )
            ax.text(
                i,
                exp_redundancy + (1 - exp_redundancy) / 2,
                "{0:.1f}%".format((1 - exp_redundancy) * 100),
                color="white",
                ha="center",
                va="center",
                fontweight="bold",
            )
        ax.set(xlabel="Configuration", ylabel="Redundancy", ylim=(0, 1.05))
        fig.tight_layout()
        return fig

    def reliability(self):
        reliabilities = pd.DataFrame(
            {
                name: exp.reliability()["reliability"].resample("1s").mean()
                for name, exp in self._experiments.items()
            }
        )
        return reliabilities.fillna(method="bfill").fillna(0).plot(title="Reliability")


ploter = PubSubPlotter({"KadPubSub": exp, "GossipSub": exp2, "Plumtree": exp3})

ploter.reliability().figure.savefig('reliability.pdf', dpi=600)
